# Setup

### Import relevant packages

In [1]:
# general packages
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import os

In [2]:
# packages for importing the cofig variables from the YAML file
import yaml

In [3]:
# packages for playing audio file
from IPython.display import Audio

### Configure max rows and max columns

In [4]:
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)

### Load in the '.py' file containg the functions used in this notebook

In [5]:
%load_ext autoreload
%autoreload 1
%aimport rf_vif_and_if_feat_select

import rf_vif_and_if_feat_select as fs

# ---------------------------------------------------------------------------------------------------------------

# Read in the variables from the YAML config file

In [6]:
# read in YAML configuration file
with open("../../Config_files/config.yaml", "r") as variables:
    config_variables = yaml.load(variables, Loader=yaml.FullLoader)

#### Read in the data directory filepath

In [7]:
data_directory = config_variables["data_directory"]

#### Read in the path to the audio file

In [8]:
audio_file_path = config_variables["audio_file_path"]

#### Read in the list of analysis types

In [9]:
types_of_analysis = config_variables["types_of_analysis"]

#### Read in the data shift periods

In [10]:
shift_periods_in_days_list = config_variables["shift_periods_in_days_list"]

#### Read the time intervals

In [11]:
train_intervals_list = config_variables["train_intervals_list"]

#### Read in the random seed

In [12]:
random_seed = config_variables["random_seed"]

# ---------------------------------------------------------------------------------------------------------------

# Read in the processed data

In [13]:
processed_data = pd.read_csv(os.path.join(data_directory, 'processed_data.csv'))

In [14]:
processed_data.shape

(4470, 1214)

In [15]:
processed_data.head()

,date,trading_volume,rank,trades_per_minute,volatility,bid_and_ask_spread,bid_and_ask_sum_asks,bid_and_ask_sum_bids,time_between_blocks,block_size_votes,marketcap3sma,marketcap7sma,marketcap14sma,marketcap30sma,marketcap90sma,marketcap3ema,marketcap7ema,marketcap14ema,marketcap30ema,marketcap90ema,marketcap3wma,marketcap7wma,marketcap14wma,marketcap30wma,marketcap90wma,marketcap3trx,marketcap7trx,marketcap14trx,marketcap30trx,marketcap90trx,marketcap3mom,marketcap7mom,marketcap14mom,marketcap30mom,marketcap90mom,marketcap3std,marketcap7std,marketcap14std,marketcap30std,marketcap90std,marketcap3var,marketcap7var,marketcap14var,marketcap30var,marketcap90var,marketcap3rsi,marketcap7rsi,marketcap14rsi,marketcap30rsi,marketcap90rsi,...,eur_jpy_exchange_open,eur_jpy_exchange_high,eur_jpy_exchange_low,eur_jpy_exchange_close,gbp_jyp_exchange_open,gbp_jyp_exchange_high,gbp_jyp_exchange_low,gbp_jyp_exchange_close,usd_gbp_exchange_open,usd_gbp_exchange_high,usd_gbp_exchange_low,usd_gbp_exchange_close,usd_eur_exchange_open,usd_eur_exchange_high,usd_eur_exchange_low,usd_eur_exchange_close,usd_cad_exchange_open,usd_cad_exchange_high,usd_cad_exchange_low,usd_cad_exchange_close,usd_aud_exchange_open,usd_aud_exchange_high,usd_aud_exchange_low,usd_aud_exchange_close,usd_mxn_exchange_open,usd_mxn_exchange_high,usd_mxn_exchange_low,usd_mxn_exchange_close,usd_hkd_exchange_open,usd_hkd_exchange_high,usd_hkd_exchange_low,usd_hkd_exchange_close,usd_jpy_exchange_open,usd_jpy_exchange_high,usd_jpy_exchange_low,usd_jpy_exchange_close,shifted_0,binary_price_change_0,shifted_1,binary_price_change_1,shifted_7,binary_price_change_7,shifted_14,binary_price_change_14,shifted_30,binary_price_change_30,shifted_60,binary_price_change_60,shifted_90,binary_price_change_90
0,2009-01-03,14378.975391,1188.702997,0.0125,787.825957,0.31782,7.608714e+07,6.866047e+07,8.141667,20.0,1.037046e+12,1.065113e+12,1.067361e+12,9.890445e+11,7.970899e+11,1.037697e+12,1.050868e+12,1.044357e+12,9.984529e+11,8.133756e+11,1.032082e+12,1.052522e+12,1.062149e+12,1.027730e+12,8.962809e+11,-0.897,-0.064,-0.19,-0.075,0.295,-3.909422e+10,-1.687704e+10,-2.127435e+09,1.035322e+10,5.972210e+11,3.344058e+10,5.552429e+10,5.852754e+10,1.667437e+11,3.772879e+11,2.795682e+20,7.707367e+20,8.563682e+20,6.950866e+21,3.558654e+22,29.393,44.274,53.277,58.277,61.742,...,113.379997,122.250000,115.949997,126.349998,130.100006,146.966003,119.239998,128.350006,0.63280,0.62410,0.76220,0.61812,0.74850,0.75850,0.891266,0.69575,1.0192,0.9815,0.9799,0.9926,1.0778,1.0805,1.0997,1.0778,13.0000,13.0000,11.591,13.0000,7.7499,7.7500,7.7491,7.7498,76.779999,77.129997,101.720001,76.860001,52184.0,1,52184.0,1,52184.0,1,52184.0,1,52184.0,1,52184.0,1,52184.0,1
1,2009-01-04,14378.975391,1188.702997,0.0125,787.825957,0.31782,7.608714e+07,6.866047e+07,8.141667,20.0,1.037046e+12,1.065113e+12,1.067361e+12,9.890445e+11,7.970899e+11,1.037697e+12,1.050868e+12,1.044357e+12,9.984529e+11,8.133756e+11,1.032082e+12,1.052522e+12,1.062149e+12,1.027730e+12,8.962809e+11,-0.897,-0.064,-0.19,-0.075,0.295,-3.909422e+10,-1.687704e+10,-2.127435e+09,1.035322e+10,5.972210e+11,3.344058e+10,5.552429e+10,5.852754e+10,1.667437e+11,3.772879e+11,2.795682e+20,7.707367e+20,8.563682e+20,6.950866e+21,3.558654e+22,29.393,44.274,53.277,58.277,61.742,...,113.379997,122.250000,115.949997,126.349998,130.100006,146.966003,119.239998,128.350006,0.63280,0.62410,0.76220,0.61812,0.74850,0.75850,0.891266,0.69575,1.0192,0.9815,0.9799,0.9926,1.0778,1.0805,1.0997,1.0778,13.0000,13.0000,11.591,13.0000,7.7499,7.7500,7.7491,7.7498,76.779999,77.129997,101.720001,76.860001,52184.0,1,52184.0,1,52184.0,1,52184.0,1,52184.0,1,52184.0,1,52184.0,1
2,2009-01-05,14378.975391,1188.702997,0.0125,787.825957,0.31782,7.608714e+07,6.866047e+07,8.141667,20.0,1.037046e+12,1.065113e+12,1.067361e+12,9.890445e+11,7.970899e+11,1.037697e+12,1.050868e+12,1.044357e+12,9.984529e+11,8.133756e+11,1.032082e+12,1.052522e+12,1.062149e+12,1.027730e+12,8.962809e+11,-0.897,-0.064,-0.19,-0.075,0.295,-3.9094

In [16]:
processed_data.tail()

,date,trading_volume,rank,trades_per_minute,volatility,bid_and_ask_spread,bid_and_ask_sum_asks,bid_and_ask_sum_bids,time_between_blocks,block_size_votes,marketcap3sma,marketcap7sma,marketcap14sma,marketcap30sma,marketcap90sma,marketcap3ema,marketcap7ema,marketcap14ema,marketcap30ema,marketcap90ema,marketcap3wma,marketcap7wma,marketcap14wma,marketcap30wma,marketcap90wma,marketcap3trx,marketcap7trx,marketcap14trx,marketcap30trx,marketcap90trx,marketcap3mom,marketcap7mom,marketcap14mom,marketcap30mom,marketcap90mom,marketcap3std,marketcap7std,marketcap14std,marketcap30std,marketcap90std,marketcap3var,marketcap7var,marketcap14var,marketcap30var,marketcap90var,marketcap3rsi,marketcap7rsi,marketcap14rsi,marketcap30rsi,marketcap90rsi,...,eur_jpy_exchange_open,eur_jpy_exchange_high,eur_jpy_exchange_low,eur_jpy_exchange_close,gbp_jyp_exchange_open,gbp_jyp_exchange_high,gbp_jyp_exchange_low,gbp_jyp_exchange_close,usd_gbp_exchange_open,usd_gbp_exchange_high,usd_gbp_exchange_low,usd_gbp_exchange_close,usd_eur_exchange_open,usd_eur_exchange_high,usd_eur_exchange_low,usd_eur_exchange_close,usd_cad_exchange_open,usd_cad_exchange_high,usd_cad_exchange_low,usd_cad_exchange_close,usd_aud_exchange_open,usd_aud_exchange_high,usd_aud_exchange_low,usd_aud_exchange_close,usd_mxn_exchange_open,usd_mxn_exchange_high,usd_mxn_exchange_low,usd_mxn_exchange_close,usd_hkd_exchange_open,usd_hkd_exchange_high,usd_hkd_exchange_low,usd_hkd_exchange_close,usd_jpy_exchange_open,usd_jpy_exchange_high,usd_jpy_exchange_low,usd_jpy_exchange_close,shifted_0,binary_price_change_0,shifted_1,binary_price_change_1,shifted_7,binary_price_change_7,shifted_14,binary_price_change_14,shifted_30,binary_price_change_30,shifted_60,binary_price_change_60,shifted_90,binary_price_change_90
4465,2021-03-26,14378.975391,1188.702997,167.461944,787.825957,0.31782,7.608714e+07,6.866047e+07,8.141667,20.0,1.037046e+12,1.065113e+12,1.067361e+12,9.890445e+11,7.970899e+11,1.037697e+12,1.050868e+12,1.044357e+12,9.984529e+11,8.133756e+11,1.032082e+12,1.052522e+12,1.062149e+12,1.027730e+12,8.962809e+11,-0.897,0.308,0.699,0.877,0.948,-3.909422e+10,-1.687704e+10,-2.127435e+09,1.035322e+10,5.972210e+11,3.344058e+10,5.552429e+10,5.852754e+10,1.667437e+11,3.772879e+11,2.795682e+20,7.707367e+20,8.563682e+20,6.950866e+21,3.558654e+22,29.393,44.274,53.277,58.277,61.742,...,128.563995,129.445007,128.529007,128.570999,150.085007,151.526001,150.035995,150.089996,0.72739,0.727620,0.72412,0.727320,0.849180,0.84943,0.84717,0.849140,1.260310,1.26046,1.256110,1.260180,1.316656,1.317800,1.309415,1.316829,20.672100,20.727301,20.532801,20.683029,7.768520,7.769900,7.768220,7.768380,109.181999,109.834000,109.139000,109.179001,52184.0,1,52184.0,1,1.0,0,1.0,0,1.0,0,1.0,0,1.0,0
4466,2021-03-27,14378.975391,1188.702997,167.461944,787.825957,0.31782,7.608714e+07,6.866047e+07,8.141667,20.0,1.037046e+12,1.065113e+12,1.067361e+12,9.890445e+11,7.970899e+11,1.037697e+12,1.050868e+12,1.044357e+12,9.984529e+11,8.133756e+11,1.032082e+12,1.052522e+12,1.062149e+12,1.027730e+12,8.962809e+11,-0.897,0.308,0.699,0.877,0.948,-3.909422e+10,-1.687704e+10,-2.127435e+09,1.035322e+10,5.972210e+11,3.344058e+10,5.552429e+10,5.852754e+10,1.667437e+11,3.772879e+11,2.795682e+20,7.707367e+20,8.563682e+20,6.950866e+21,3.558654e+22,29.393,44.274,53.277,58.277,61.742,...,128.839330,129.426671,128.630335,128.851334,150.500005,151.615402,150.213999,150.525996,0.72666,0.727377,0.72348,0.726583,0.848903,0.84969,0.84742,0.848867,1.259687,1.26112,1.256673,1.259593,1.314185,1.316208,1.308377,1.314336,20.658696,20.746587,20.548700,20.666196,7.768647,7.771633,7.768443,7.768537,109.380666,109.815666,109.220668,109.383001,52184.0,1,52184.0,1,1.0,0,1.0,0,1.0,0,1.0,0,1.0,0
4467,2021-03-28,14378.975391,1188.702997,167.461944,787.825957,0.31782,7.608714e+07,6.866047e+07,8.141667,20.0,1.037046e+12,1.065113e+12,1.067361e+12,9.890445e+11,7.970899e+11,1.037697e+12,1.050868e+12,1.044357e+12,9.984529e+11,8.133756e+11,1.032082e+12,1.052522e+12,1.062149e+12,1.027730e+12,

# ---------------------------------------------------------------------------------------------------------------

# Iterate through the intervals, # features and shift periods & select the most important features from the data

In [17]:
plot_data = True
print_vif_outputs = True
print_outliers = True
reg_perc_to_retain = 0.71
clas_perc_to_retain = 0.77

In [20]:
for reg_or_clas in tqdm(types_of_analysis):
    print("==========\n  ", reg_or_clas, "  \n==========")
    
    # set the percentage of feature importance to retain
    percent_importance_to_retain = reg_perc_to_retain if reg_or_clas == 'reg' else clas_perc_to_retain
    
    for interval_num in range(1, len(train_intervals_list)+1):
        print("-------------")
        print("Interval:", interval_num)

        # get the start and end dates for this interval
        train_start_date, train_end_date = train_intervals_list[interval_num-1]

        # define the train data for this date interval
        train_df = fs.get_subset_of_data(processed_data, train_start_date, train_end_date)
        fs.get_date_range_of_data(train_df) 

        # iterate over the shift periods and select the features based on this period
        for shift_period in shift_periods_in_days_list:

            # get a dataframe of the data's features
            train_features, train_cols_to_drop, train_colname = fs.get_features(train_df, shift_period, reg_or_clas)
        
            # get a numpy array of the feature we are trying to predict
            y = np.ravel(train_features[train_colname])

            # use Random Forest model and Variance Inflation Factor (VIF) to get the most important columns
            dropped_merged_df = fs.use_rand_forests_and_vif_to_subset_features(reg_or_clas, train_features, y, config_variables, train_colname, print_vif_outputs)

            if 'price' not in dropped_merged_df.columns:
                dropped_merged_df['price'] = train_features['price']
            
            # Use random forest to score feature importance - then only keep specified % importance
            selected_features_df = fs.further_subset_features_using_feat_importance(reg_or_clas, dropped_merged_df, y, percent_importance_to_retain, plot_data, random_seed)

            # add the created price column back into the data
            selected_features_df["_".join(train_colname.split('_')[:-1])] = y

            # Remove the outliers in the data using an Isolation forest
            final_df = fs.remove_outliers(selected_features_df, y, reg_or_clas, print_outliers)

            # Output the data to a CSV
            train_path = os.path.join(data_directory, 'Feat_select', '{}_rf_vif_and_if_interval_{}_shift_{}.csv'.format(reg_or_clas, interval_num, shift_period))
            final_df.to_csv(train_path, index=False)

# play an audio file to tell me that this feature selection has finished
Audio(audio_file_path, rate=5, autoplay=True)

   clas   
-------------
Interval: 1
 - This data conatins 2160 days from '2015-02-01' to '2020-12-31'



1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 
---Indicators---
Dropped:
 ['confirmationtime14ema', 'confirmationtime3ema', 'confirmationtime30wma', 'size90rsi', 'confirmationtime7ema', 'confirmationtime3wma', 'confirmationtime90sma', 'tweets14rsi', 'size14sma', 'confirmationtime30sma', 'tweets30ema', 'hashrate30rsi', 'sentbyaddress30std', 'size7wma', 'tweets90ema', 'confirmationtime3sma', 'activeaddresses90wma', 'transactions14std', 'size3std', 'transactionvalue7std', 'size3mom', 'transactions90std', 'size90mom', 'sentbyaddress30var', 'tweets7rsi', 'top100cap30roc', 'tweets7sma', 'sentbyaddress14var']
Remaining variables:
 ['confirmationtime7mom', 'google_trends7trx', 'hashrate3trx', 'marketcap3rsi', 'median_transaction_fee90trx', 'size3roc', 'size3var', 'size90roc', 'size90var', 'top100cap14roc', 'transactionvalue14trx', 'transactionvalue14wma', 'transactionvalue30trx', 'transactionvalue7var', 'tweets14mom', 'tweets30mom', 'tweets7roc']


C:\Users\Owner\.conda\envs\python_all\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\Owner\.conda\envs\python_all\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 


KeyboardInterrupt: 